### Import Library dan CSV

In [41]:
import pandas as pd
import re
import string

from indoNLP.preprocessing import replace_word_elongation, replace_slang

In [42]:
angerData = pd.read_csv('./data/AngerData.csv', delimiter='\t')
fearData = pd.read_csv('./data/FearData.csv', delimiter='\t')
joyData = pd.read_csv('./data/JoyData.csv', delimiter='\t')
loveData = pd.read_csv('./data/LoveData.csv', delimiter='\t')
sadData = pd.read_csv('./data/SadData.csv', delimiter='\t')\
    
#gabungkan data
df = pd.concat([angerData, fearData, joyData, loveData, sadData], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True) #shuffle data

df.head()

,Tweet,Label
0,"kadang, meski marah atas apa yang telah dilaku...",Love
1,musiknya sedih,Sad
2,gue ga suka banget dichat terus.. ya kalo pemb...,Anger
3,ketika lu punya pacar tapi tbtb ada seseorang ...,Fear
4,alhamdulilah jimin kemarin mimpi jimin dan sek...,Joy


In [43]:
# Mengubag nama label menjadi bahasa Indonesia
label_mapping = {
    'Anger': 'Marah',
    'Fear': 'Takut',
    'Joy': 'Gembira',
    'Love': 'Cinta',
    'Sad': 'Sedih'
}
df['Label'] = df['Label'].map(label_mapping)
df.head()

,Tweet,Label
0,"kadang, meski marah atas apa yang telah dilaku...",Cinta
1,musiknya sedih,Sedih
2,gue ga suka banget dichat terus.. ya kalo pemb...,Marah
3,ketika lu punya pacar tapi tbtb ada seseorang ...,Takut
4,alhamdulilah jimin kemarin mimpi jimin dan sek...,Gembira


In [44]:
with pd.option_context('display.max_colwidth', None):
    print(df.iloc[4])

Tweet    alhamdulilah jimin kemarin mimpi jimin dan sekarang dapat kabar kekgini seneng banget jimin we love you
Label                                                                                                    Gembira
Name: 4, dtype: object


### Cleaning Data

In [45]:
def cleanDataframe(df):
    print('Missing Values:', df.isnull().sum())
    print('Duplicates:', df.duplicated().sum())

    df_cleaned = df.dropna(subset='Tweet')
    df_cleaned = df_cleaned.drop_duplicates(keep='first')

    print('\nMissing values after cleaning:', df_cleaned.isnull().sum())
    print('Duplicates after cleaning:', df_cleaned.duplicated().sum())
    return df_cleaned




def cleaningText(text):
    # Membersihkan tanda tanda sisa medsos
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Menghapus Mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text)  # Menghapus Hashtag
    text = re.sub(r'RT[\s]', '', text)  # menghapus RT
    text = re.sub(r"http\S+", '', text)  # menghapus link

    # Pembersihan Karakter
    
    text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))# Mengganti tanda baca dengan spasi (alih-alih menghapus)
    text = re.sub(r'\d+', '', text)  # Menghapus angka
    text = text.replace('\n', ' ')  # Mengganti garis baru dengan spasi
    return text


kamusCleaning = pd.read_csv('kamus.csv', encoding='utf-8',
                            delimiter=',', header=None, names=['slang', 'formal'])
cleaningDict = dict(zip(kamusCleaning.slang, kamusCleaning.formal))


def applyCleaningDict(text):
    # Membersihkan kata yang tidak terbersihkan dari kamus IndoNLP dari kamus tambahan
    words = text.split()
    cleaned_words = [cleaningDict[word]
                     if word in cleaningDict else word for word in words]
    cleaned_text = ' '.join(cleaned_words)
    return cleaned_text

def normalizeText(text):
    text = text.strip(' ')  # Menghapus karakter spasi dari kiri dan kanan teks
    text = text.lower()  # mengubah semua karakter dalam teks menjadi huruf kecil
    text = re.sub(r'(.)\1+', r'\1\1', text) # Menghapus text elongation (library indoNLP)
    text = replace_word_elongation(text) # Menghapus huruf berlebih di belakang
    text = replace_slang(text)  # Menghapus slangwords dari kamus IndoNLP
    text = applyCleaningDict(text) # Membersihkan kata yang tidak terbersihkan dari kamus IndoNLP dari kamus tambahan
    text = re.sub(r'\s+', ' ', text) # Mengganti multiple spasi dengan satu spasi
    
    return text

In [46]:
df_cleaned = cleanDataframe(df)

Missing Values: Tweet    0
Label    0
dtype: int64
Duplicates: 42

Missing values after cleaning: Tweet    0
Label    0
dtype: int64
Duplicates after cleaning: 0


In [47]:
df_cleaned['Tweet'] = df_cleaned['Tweet'].apply(cleaningText)
# Print 1 sample not truncated
with pd.option_context('display.max_colwidth', None):
    print(df_cleaned.iloc[2])


Tweet    gue ga suka banget dichat terus   ya kalo pembahasan lo bagus dan menarik bakal gue tanggepin  masalahnya pembahasan lo tuh itu  aja dan hal  yang lo bahas tuh hal  yang menurut gue sensitif   jadi ya gue males balas chat lo
Label                                                                                                                                                                                                                               Marah
Name: 2, dtype: object


In [48]:
all_text = "".join(df_cleaned["Tweet"].astype(str))
text_length = all_text.split()
unique_text = set(text_length)
print('jumlah kata unik sebelum normalisasi:', len(unique_text))

jumlah kata unik sebelum normalisasi: 15266


In [49]:
df_normalized = df_cleaned.copy()
df_normalized['Tweet'] = df_normalized['Tweet'].apply(normalizeText)
# Print 1 sample not truncated
with pd.option_context('display.max_colwidth', None):
    print(df_normalized.iloc[2])

Tweet    saya tidak suka sekali dichat terus ya kalau pembahasan kamu bagus dan menarik bakal saya tanggepin masalahnya pembahasan kamu tuh itu saja dan hal yang kamu bahas tuh hal yang menurut saya sensitif jadi ya saya malas balas chat kamu
Label                                                                                                                                                                                                                                        Marah
Name: 2, dtype: object


In [50]:
all_text_normalized = "".join(df_normalized["Tweet"].astype(str))
text_length_normalized = all_text_normalized.split()
unique_text_normalized = set(text_length_normalized)
print('jumlah kata unik setelah normalisasi:', len(unique_text_normalized))

jumlah kata unik setelah normalisasi: 14466


In [51]:
print(df_cleaned['Label'].value_counts())

Label
Gembira    1254
Marah      1043
Sedih       995
Takut       906
Cinta       754
Name: count, dtype: int64


### Export Menjadi CSV Baru

In [52]:
# Export
df_cleaned.to_csv('./data/data_cleaned.csv', index=False)